In [1]:
import pandas as pd
import textstat

In [2]:
from bs4 import BeautifulSoup

In [3]:
from tqdm._tqdm_notebook import tqdm_notebook

C:\Users\alecr\Anaconda3\envs\gensim_env\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """Entry point for launching an IPython kernel.


In [4]:
from langdetect import detect

In [5]:
import seaborn as sns

In [6]:
%config Completer.use_jedi = False

In [7]:
df = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv")

df.drop(['Unnamed: 0'], axis=1, inplace=True)

C:\Users\alecr\Anaconda3\envs\gensim_env\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (20,23,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.head()

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,slug,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,garry-breitkreuz-1,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3


In [9]:
df.columns

Index(['id', 'politician_id', 'riding_id', 'party_id', 'end_date',
       'start_date', 'id.1', 'document_id', 'time', 'h1_en', 'h2_en',
       'member_id', 'who_en', 'content_en', 'sequence_en', 'wordcount',
       'politician_id.1', 'procedural', 'h3_en', 'who_hocid', 'content_fr',
       'statement_type', 'written_question', 'source_id', 'who_context_en',
       'slug', 'urlcache', 'h1_fr', 'h2_fr', 'h3_fr', 'who_fr',
       'who_context_fr', 'wordcount_en', 'tokenized_content', 'slug_length'],
      dtype='object')

In [10]:
content = df.tokenized_content

In [11]:
%%time
df['soup'] = df.content_en.apply(lambda x: BeautifulSoup(x).text)

Wall time: 3min 18s


In [12]:
df['text_len'] = df.soup.apply(len)

In [13]:
df = df[df.text_len>2]

In [14]:
content[0]

"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a', 'point', 'of', 'order', '.', 'i', 'believe', 'you', 'would', 'find', 'unanimous', 'consent', 'following', 'consultation', 'among', 'all', 'parties', 'to', 'further', 'defer', 'the', 'vote', 'just', 'deferred', 'until', 'monday', 'to', 'next', 'tuesday', 'at', 'the', 'end', 'of', 'government', 'orders', '.']"

In [15]:
import ast

In [16]:
ast.literal_eval(content[0])

['mr.',
 'speaker',
 ',',
 'i',
 'rise',
 'on',
 'a',
 'point',
 'of',
 'order',
 '.',
 'i',
 'believe',
 'you',
 'would',
 'find',
 'unanimous',
 'consent',
 'following',
 'consultation',
 'among',
 'all',
 'parties',
 'to',
 'further',
 'defer',
 'the',
 'vote',
 'just',
 'deferred',
 'until',
 'monday',
 'to',
 'next',
 'tuesday',
 'at',
 'the',
 'end',
 'of',
 'government',
 'orders',
 '.']

%%time
df['tokens'] = df.tokenized_content.apply(ast.literal_eval) 

In [17]:
%%time
df['reading_ease'] = df.soup.apply(textstat.flesch_reading_ease)

Wall time: 8min 13s


In [18]:
%%time
df['fk_reading_ease'] = df.soup.apply(textstat.flesch_kincaid_grade)

KeyboardInterrupt: 

In [21]:
def ld (row):
    return detect(row['soup'])

In [22]:
df.fk_reading_ease.describe()

AttributeError: 'DataFrame' object has no attribute 'fk_reading_ease'

In [23]:
maybefrench = df[df.reading_ease<-2000] 

In [ ]:
df[df.text_len<2]

In [ ]:
df[df.text_len<5]

I'm assuming that if the reading ease is above 20, it's in English, for now

In [ ]:
english = df[df.reading_ease>20] 

In [ ]:
english.shape

In [ ]:
english.sort_values(by='reading_ease', ascending=False)

In [ ]:
reading_ease_party = df.pivot_table(values='reading_ease', index = "party_id", aggfunc='mean')

In [ ]:
reading_ease_party.sort_values(by='reading_ease', ascending=False, inplace=True)

In [ ]:
reading_ease_party

In [ ]:
party_df = pd.read_csv(r"D:\data\openparliament\core_party.csv", sep='\t', header=None)
party_names = ['id', 'name_en', 'slug','short_name_en', 'name_fr', 'short_name_fr']
party_df.columns = party_names

In [ ]:
reading_ease_party = reading_ease_party.merge(party_df, how='inner', left_on='party_id', right_on='id')

In [ ]:
reading_ease_party

In [ ]:
fk_party = english.pivot_table(values='fk_reading_ease', index = "party_id", aggfunc='mean')

In [ ]:
fk_party =  fk_party.merge(party_df, how='inner', left_on='party_id', right_on='id')

In [ ]:
fk_party.sort_values(by='fk_reading_ease',ascending=False, inplace=True)

In [ ]:
fk_party

In [ ]:
reading_ease_politician = df.pivot_table(values='reading_ease', index = "politician_id", aggfunc='mean')

In [ ]:
df_electedmember = pd.read_csv(r"D:\data\openparliament\politician_info.csv")

In [ ]:
df_electedmember.head()

In [ ]:
reading_ease_politician = reading_ease_politician.merge(df_electedmember, how='inner', left_on='politician_id', right_on='politician_id')

In [ ]:
reading_ease_politician.sort_values(by='reading_ease', ascending=False, inplace=True)

In [ ]:
#reading_ease_politician.head()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
reading_ease_party.slug.fillna('', inplace=True)

In [ ]:
party_colours = ['blue', 'blue', 'blue', 'turquoise', 'blue','orange','grey',
                'red', 'green', 'black', 'turquoise']

In [ ]:
reading_ease_party.loc[3,'slug'] = 'QD'

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Flesch Reading Ease by Party')
plt.xlabel('party name slug')
plt.ylabel('Flesch Reading Ease score')
plt.bar(reading_ease_party.slug, reading_ease_party.reading_ease, 
        color=party_colours)

plt.show()
#plt.savefig(r'C:\Users\alecr\Projects\openparliament-NLP\word_stats\flesch_by_party.png')

In [ ]:
party_colour_dict = {28:'blue',25: 'blue', 1:'blue', 46:'turquoise', 26: 'blue',2: 'orange',5: 'grey',
                4: 'red', 9: 'green', 3: 'black', 39: 'turquoise'}

In [35]:
reading_ease_party['party_colour'] = reading_ease_politician.party_id.replace(party_colour_dict)

NameError: name 'reading_ease_politician' is not defined

In [ ]:
plt.scatter(reading_ease_politician.party_id, reading_ease_politician.reading_ease,c=reading_ease_politician.party_colour)
plt.xlabel('party id')
plt.show()

In [ ]:
reading_ease_politician.sort_values(by='reading_ease', ascending=True)

In [ ]:
df_environment = df[df.h2_en=='The Environment']

In [ ]:
fk_politician = english.pivot_table(values='fk_reading_ease', index = "politician_id", aggfunc='mean')

In [ ]:
fk_politician = fk_politician.merge(df_electedmember, how='inner', left_on='politician_id', right_on='politician_id')

In [ ]:
fk_politician['party_colour'] = fk_politician.party_id.replace(party_colour_dict)

In [ ]:
plt.scatter(fk_politician.party_id, fk_politician.fk_reading_ease,c=fk_politician.party_colour)
plt.xlabel('party id')
plt.ylabel('flesh-kincaid grade level')
plt.show()

In [ ]:
plt.scatter(fk_politician.start_date, fk_politician.fk_reading_ease, c=fk_politician.party_colour)
plt.xlabel('politician start date')
plt.ylabel('flesch-kincaid grade level')
plt.legend(fk_politician.party_colour)
plt.show()

In [ ]:
fk_politician.shape

In [ ]:
df_environment.shape

In [ ]:
df_environment.pivot_table(values='reading_ease',index='party_id',aggfunc='mean').sort_values(by='reading_ease', ascending=False)

In [ ]:
def getReadingEaseBy_h2_Topic(topic):
    topic_pivot = df[df.h2_en==topic].pivot_table(values='reading_ease',index='party_id',aggfunc='mean')
    topic_pivot.sort_values(by='reading_ease', ascending=False, inplace = True)
    return topic_pivot

In [ ]:
getReadingEaseBy_h2_Topic('Committees of the House')

In [ ]:
df_carbon = df[df.h2_en.str.contains('Carbon')]

In [ ]:
df_carbon.pivot_table(values='reading_ease', index='party_id', aggfunc='mean').sort_values(by='reading_ease', ascending=False)

In [ ]:
from textblob import TextBlob

In [ ]:
TextBlob(df_carbon.soup.iloc[0]).sentiment

In [ ]:
df_carbon['sentiment'] = df_carbon.soup.apply(lambda x: TextBlob(x).sentiment[0])

In [ ]:
df_carbon['subjectivity'] = df_carbon.soup.apply(lambda x: TextBlob(x).sentiment[1])

In [ ]:
green_sentiments = df_carbon[df_carbon.party_id==9].sentiment

In [ ]:
df[df.h3_en =='Cancellation of Teck Frontier Mine Project'].soup

In [ ]:
df.columns

In [ ]:
df['end_date'][0]

In [ ]:
df['start_date'][0]

In [ ]:
sentiments = df.groupby(['h2_en', 'party_id'])

for h in df.h2_en[:50]:
    print('\n',h)

In [ ]:
type(sentiments)

note: save with soup, eliminate unnecessary columns, processing every time is ridiculous

In [33]:
env_flesch = df[df.h2_en.str.contains('environment')].pivot_table(index='party_id', values='reading_ease', aggfunc='mean')

In [34]:
base_flesch = df.pivot_table(index='party_id', values='reading_ease', aggfunc='mean')

In [ ]:
comparison = base_flesch.merge(env_flesch)